### Softmax Regression Model

Here we are going to make an MNIST model using softmax regression. This will essentially be a one layer neural network.

#### Imports

First we will do our imports. TensorFlow has a utility that will handle the downloading and reading of the MNIST data automatically, so we will be using that. Next we need to import tensorflow itself, so that we can use it.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

# Import additional libraries for visualization.
import numpy as np
import matplotlib.pyplot as plt
from random import randint

%matplotlib inline

tf.logging.set_verbosity(tf.logging.ERROR)

#### Read data
input_data is a utility function provided by TensorFlow to retrieve MNIST dataset. The images downloaded aren’t in any standard image format. They are in a special format called the IDX file format which is a simple format for vectors and multidimensional matrices of various numerical types.

One_hot refers to how the labels will be represented: as **one-hot** vectors.

A one-hot vector is a vector which is 0 in most dimensions, and 1 in a single dimension. 
E.g.  3 = [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

The reason we one-hot encode, is not only to convert labels to a numerical representation, but also to provide the machine learning algorithms a format easier to work with and provide more expressive representations of the data. 
In the case of classification, encoding one-hot vectors allows us to learn and represent a  probability-like number for all the label values.


In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

### How is the MNIST data represented?

In [ ]:
print ("Shape of training images tensor is %s" % (str(mnist.train.images.shape)))
print ("Shape of training labels tensor is %s" % (str(mnist.train.labels.shape)))
print ("Shape of testing images tensor is %s" % (str(mnist.test.images.shape)))
print ("Shape of testing labels tensor is %s" % (str(mnist.test.labels.shape)))

In [ ]:
# Print a random image in the training set to see how it looks.
rand_index = randint(0, mnist.train.images.shape[0] - 1)

label = mnist.train.labels[rand_index, :]
print("How a label is represented. The index corresponds to the actual number:")
print(str(label) + '\n')
image = mnist.train.images[rand_index, :]
selected_img = np.reshape(image, (28, 28)) # 28 by 28 matrix 

selected_label = np.argmax(label)
plt.matshow(selected_img, cmap=plt.get_cmap('gray_r'))
print("Training image %s has label %s" % (str(rand_index), str(selected_label)))

### Sample visual image representation

```
0.0 0.0 0.0 0.0 ... 0.0 0.0
0.0 0.0 0.2 0.3 ... 0.3 0.0
0.0 0.0 0.8 0.8 ... 0.4 0.0
 |   |   |   |  ...  |   |
0.0 0.0 0.5 0.7 ... 0.4 0.0
0.0 0.0 0.0 0.2 ... 0.8 0.0 
```

An image can be interpreted as a 28x28 array of floats from 0.0 to 1.0. Numbers closer to 1.0 denote higher intensity/blackness. We flatten this array into a vector of 28x28 = 784 length.

#### Create placeholders

Placeholder is used to feed actual training examples. We will feed image and label data to these when we have TensorFlow run computations. 

A placeholder is simply a variable that we will assign data to at a later date. It allows us to create our operations and build our computation graph, without needing the data. In TensorFlow terminology, we then feed data into the graph through these placeholders.

We want to be able to input any number of MNIST images, each flattened into a 784-dimensional vetor. We represent this as a 2D tensor of floating-point numbers, with shape [None, 784]. Here None means that a dimension can be of any length.



In [ ]:
# Placeholder that will be fed image data.
x = tf.placeholder(tf.float32, [None, 784])
# Placeholder that will be fed the correct labels.
y_ = tf.placeholder(tf.float32, [None, 10])

#### Weight and Bias Variable Functions

So as we progress we will be making a lot of bias and weight variables for the model to learn, so we will make functions for these.

Breaking symmetry is essential here.

As you go deep, it becomes important to initialise weights with random values. The optimiser can get stuck in its initial position if you do not. One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients

Starting from any fixed initialization biases your solution towards some one particular set of weights. If you do it randomly (and possibly many times) then it is much less probable that you will get stuck in some weird part of the error surface.

tf.truncated_normal is a TensorFlow function that produces random values following the normal (Gaussian) distribution between -2*stddev and +2*stddev. So we will initialize weight variables with that.

Similarly, we initialise biases with small positive values to avoid too many dead neurons.

These are more for the more complex model we will make later, as initializing the weights and biases with zeros for this simple softmax regression model would be fine.

#### Define variables.

Variables are modifiable tensors that lives in TensorFlow's graph of interacting operations. These are what we will trying to find optimal values for during our training.

Notice that W has a shape of [784, 10] because we want to multiply the 784-dimensional image vectors by it to produce 10-dimensional vectors of evidence for the difference classes. b has a shape of [10] so we can add it to the output

In [ ]:
def weight_variable(shape):
    """Generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    """Generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Define weight and bias.
W = weight_variable([784, 10])
b = bias_variable([10])

In [ ]:
print(W)

#### Define our model.

We will base our model off of a softmax regression. Softmax is good for assigning probabilities to an object being one of several things.

A softmax regression has two steps:
 1. Add up the evidence of our input being in certain classes
 2. Convert that evidence into probabilities.
 
To add up the evidence that a given image is in a particular class, we do a weighted sum of the pixel intensities. The weight is negative if that pixel having a high intensity is evidence against the image being in that class, and positive if it is evidence in favor.

But it's often more helpful to think of softmax the first way: exponentiating its inputs and then normalizing them. The exponentiation means that one more unit of evidence increases the weight given to any hypothesis multiplicatively. And conversely, having one less unit of evidence means that a hypothesis gets a fraction of its earlier weight.

This is what the function is normally used for: to highlight the largest values and suppress values which are significantly below the maximum value

The sum of all the outputs of softmax will equal to 1.

Extra evidence called bias is added. This provides the model with a trainable value in addition to the normal inputs as some things are more likely independent of the inputs.


In [ ]:
 y = tf.nn.softmax(tf.matmul(x, W) + b)

#### Define our loss.

We need to define what it means for our model to be bad. This is the loss, or cost, and it represents how far off our model is from the desired outcome.

A commonly used function for determining loss of a model is cross entropy. Here, we use the raw formulation for cross entropy.

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

#### Define our optimizer.
Now we aim to train our model using the gradient descent algorithm with a learning rate of 0.5. Gradient descent is a simple procedure, where TensorFlow simply shifts each variable a little bit in the direction that reduces the cost.

Learning rate refers to the size of the adjustments made to the weights with each iteration, the step size. A high learning rate makes a net traverse the errorscape quickly, but also makes it prone to overshoot the point of minimum error. A learning rate that is too large can hinder convergence and cause the loss function to fluctuate around the minimum or even to diverge.

A low learning rate is more likely to find the minimum, but it will do so very slowly, because it is taking small steps in adjusting the weights.

In this step, TensorFlow computes the partial derivatives of the loss function relatively to all the weights and all the biases (the gradient).

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

#### Define accuracy

tf.argmax(y, 1) is the label our model thinks is most likely for each input, while tf.argmax(y_, 1) is the correct label. We can use tf.equal to check if our prediction matches the truth. This gives us a list of booleans. To determine what fraction are correct, we cast to floating point numbers and then take the mean.

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)
accuracy2 = tf.reduce_sum(correct_prediction)

#### Launch session

A Session object encapsulates the environment in which Operation objects are executed, and Tensor objects are evaluated. To actually evaluate the nodes, we must run the computational graph within a session. A session encapsulates the control and state of the TensorFlow runtime.

In [ ]:
sess = tf.InteractiveSession()

#### Initialize variables

An Op that initializes global variables in the graph. This always needs to be run before training.

In [ ]:
tf.global_variables_initializer().run()

#### Do the training

Here, we run our training step 1100 times, feeding in batches of data to replace the placeholders.
The batches are random data points we retrieve from our image training set.
We then check the model with the test data to get our overall accuracy.

Using small batches of random data is called stochastic training -- in this case, stochastic gradient descent. Ideally, we'd like to use all our data for every step of training because that would give us a better sense of what we should be doing, but that's expensive. So, instead, we use a different subset every time. Doing this is cheap and has much of the same benefit.

The smaller the batch the less accurate estimate of the gradient. Batch size of 1 adds instability to the learning process as the weights widely vary with each batch. Weight updates happen after each batch propogates through the model.

In [ ]:
for i in range(1100):
    batch = mnist.train.next_batch(100)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1]})
        print("Step %d, Training Accuracy %g" % (i, float(train_accuracy)))
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

# See how model did.
print("Test Accuracy %g" % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

### See sample predictions

In [ ]:
print('P: Prediction, L: Actual Label')
for i in range(10):
    # Random Image index
    rand_index = randint(0, mnist.train.images.shape[0] - 1)

    x_train = mnist.train.images[rand_index,:].reshape(1,784)
    y_train = mnist.train.labels[rand_index,:]
    label = y_train.argmax()
    prediction = sess.run(y, feed_dict={x: x_train})
    print(prediction)
    plt.subplot(2, 5, i+1)
    prediction = prediction.argmax()
    plt.title('P: %d  L: %d' % (prediction, label))
    frame = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
    plt.imshow(x_train.reshape([28,28]), cmap=plt.get_cmap('gray_r'))

### Extra: Visualize the weights the model learns for each digit

Red represents negative weights, while blue represents positive weights.

In [ ]:
for i in range(10):
    plt.subplot(2, 5, i+1)
    weight = sess.run(W)[:,i]
    plt.title(i)
    plt.imshow(weight.reshape([28,28]), cmap=plt.get_cmap('seismic_r'), interpolation='nearest')
    frame = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)